# Redes Neurais

No exemplo abaixo é mostrado como fazer o treinamento de uma rede neural bem simples para classificar o dataset MNIST, o dataset MNIST é um conjunto de dígitos escritos a mão de 0 a 9.

In [16]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

## carrega o dataset mnist
root = './data'
if not os.path.exists(root):
    os.mkdir(root)
    
# function utilizado para fazer a normalização
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])

# carrega o dataset MNIST, caso esse não exista, faz o download
train_set = dset.MNIST(root=root, train=True, transform=trans, download=True)

batch_size = 100

# instancia o dataloader utilizando o dataset MNIST
train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)

print (' -- total trainning batch number: {0}'.format(len(train_loader)))

 -- total trainning batch number: 600
 -- total testing batch number: 100


`dset.MNIST` carrega o dataset MNIST, caso esse ainda não exista, esse método faz o download. O argumento trans é um function utilizado para normalizar os dados do MNIST.
`DataLoader` instancia o dataload que será utilizado durante o treinamento

In [20]:
# definição do modelo
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = MyNet()
print(model)

MyNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


Para definir o seu modelo no PyTorch você deve criar uma classe que herda de `nn.Module`. O design do PyTorch é fortemente baseado em orientação a objetos.
A classe do seu modelo precisa implementar o método  `forward`, assim o esqueleto da classe que representa o nosso modelo é:
```
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        
    def forward(self, x):
        pass
```
No método `__ini__` são instanciados os objetos utilizados no modelo, nesse caso as camadas que serão utilizadas no modelo. No nosso caso serão utilizadas 3 camadas lineares.

`torch.nn.Linear(in_features, out_features, bias=True)` essa é a assinatura do objeto Linear, o primeiro argumento é número de features de entrada, o segundo é o número de features de saída e o terceiro especifica se terá ou não bias. Como sabemos o tamanho das imagens do MNIST é 28x28 pixels monocromáticos, ou seja, apensa em tons de cinza.

O método `forward` é onde essas camadas são conectadas. Uma atenção bem grande deve ser dada as conexões das camadas, o argumento `x` representa a entrada do modelo, é por ele que irá fluir o nosso tensor de entrada.

`x = x.view(-1, 28*28)`: O método `view` modifica o shape do tensor, e no caso o argumento -1 diz para o método que nós não sabemos quantas linhas tem o nosso tensor, note que apenas um parametro pode -1 no método `view`.

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    # trainning
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        x, target = Variable(x), Variable(target)
        out = model(x)
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print("exec loop: ", batch_idx, '-', target)
        if (batch_idx+1) % 100 == 0 or (batch_idx+1) == len(train_loader):
            print ('==>>> epoch: {0}, batch index: {1}, train loss: {2}'.format(
                epoch, batch_idx+1, loss.item()))